### Машинное Обучения

## Домашнее задание №2 - Дерево решений

**Общая информация**

**Срок сдачи:** 23 апреля 2024, 23:59   
**Штраф за опоздание:** -2 балла за каждые 2 дня опоздания

Решений залить в свой github репозиторий.

Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс MyDecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict . (Но название класса не менять)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = dict()
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """

        if self.criterion == 'misclassification_error':
            min_loss = 10 ** 18
            optimal_border = None
            for j in range(x.shape[1]):
                for t in x[:, j]:
                    x_l, x_r, y_l, y_r = self.__div_samples(x, y, j, t)
                    p = dict()
                    for i in range(len(y_l)):
                        if y_l[i] in p:
                            p[y_l[i]] += 1
                        else:
                            p[y_l[i]] = 1
                    if len(y_l) != 0:
                        p_l = max(p) / len(y_l)
                        H_l = 1 - p_l
                    else:
                        H_l = 0
                    p = dict()
                    for i in range(len(y_r)):
                        if y_r[i] in p:
                            p[y_r[i]] += 1
                        else:
                            p[y_r[i]] = 1
                    if len(y_r) != 0:
                        p_r = max(p) / len(y_r)
                        H_r = 1 - p_r
                    else:
                        H_r = 0
                    loss = len(y_l) * H_l + len(y_r) * H_r
                    if loss < min_loss:
                        min_loss, optimal_border = loss, (j, t)
            return optimal_border

        if self.criterion == 'entropy':
            min_loss = 10 ** 18
            optimal_border = None
            classes, freq = np.unique(y, return_counts=True)
            p = freq / len(y)
            H = - np.dot(p, np.log(p))
            for j in range(x.shape[1]):
                for t in x[:, j]:
                    x_l, x_r, y_l, y_r = self.__div_samples(x, y, j, t)
                    classes, freq = np.unique(y_l, return_counts=True)
                    p = freq / len(y_l)
                    H_l =  -np.dot(p, np.log(p))
                    classes, freq = np.unique(y_r, return_counts=True)
                    p = freq / len(y_r)
                    H_r = -np.dot(p, np.log(p))
                    loss = len(y_l) * H_l + len(y_r) * H_r
                    if loss < min_loss:
                        min_loss, optimal_border = loss, (j, t)
            return optimal_border, H - min_loss / len(y)

        if self.criterion == 'gini':
            min_loss = 10 ** 18
            optimal_border = None
            classes, freq = np.unique(y, return_counts=True)
            p = freq / len(y)
            H = np.dot(p, 1 - p)
            for j in range(x.shape[1]):
                for t in x[:, j]:
                    x_l, x_r, y_l, y_r = self.__div_samples(x, y, j, t)
                    classes, freq = np.unique(y_l, return_counts=True)
                    p = freq / len(y_l)
                    H_l = np.dot(p, 1 - p)
                    classes, freq = np.unique(y_r, return_counts=True)
                    p = freq / len(y_r)
                    H_r = np.dot(p, 1 - p)
                    loss = len(y_l) * H_l + len(y_r) * H_r
                    if loss < min_loss:
                        min_loss, optimal_border = loss, (j, t)
            return optimal_border, H - min_loss / len(y)


    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        if len(y) == 0:
            return
        if depth > self.max_depth or len(y) < self.min_samples_split:
          classes, freq = np.unique(y, return_counts=True)
          max_f = max(freq)
          ind = np.where(freq == max_f)[0][0]
          self.tree[node_id] = (self.__class__.LEAF_TYPE, classes[ind], max_f / len(y))
        else:
            border, gain = self.__find_threshold(x, y)
            if border[0] in self.feature_importances_:
              self.feature_importances_[border[0]] += gain
            self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, border[0], border[1])
            x_l, x_r, y_l, y_r = self.__div_samples(x, y, border[0], border[1])
            if len(y_l) != 0:
              self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
            if len(y_r) != 0:
              self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        if node_id in self.tree:
            node = self.tree[node_id]
            if node[0] == self.__class__.NON_LEAF_TYPE:
                _, feature_id, threshold = node
                if x[feature_id] > threshold:
                    return self.__predict_class(x, 2 * node_id + 1)
                else:
                    return self.__predict_class(x, 2 * node_id + 2)
            else:
                return node[1]

    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def get_feature_importance():
        """
        Возвращает важность признаков
        """
        # Ваш код здесь
        pass

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [5]:
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [6]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.8888888888888888
0.9444444444444444


Совет: Проверьте, что ваша реализация корректно работает с признаками в которых встречаются повторы.
И подумайте, какие еще граничные случаи могут быть.
Например, проверьте, что на таком примере ваша модель корректно работает:

In [7]:
X = np.array([[1] * 10, [0, 1, 2, 5, 6, 3, 4, 7, 8, 9]]).T
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
for depth in range(1, 5):
    my_clf = MyDecisionTreeClassifier(max_depth=depth)
    my_clf.fit(X, y)
    print("DEPTH:", depth, "\n\t\tTree:", my_clf.tree, my_clf.predict(X))

DEPTH: 1 
		Tree: {0: (0, 1, 2), 1: (0, 1, 6), 3: (1, 1, 1.0), 4: (1, 0, 0.5), 2: (0, 0, 1), 6: (1, 0, 1.0)} [0 0 0 0 0 0 0 1 1 1]
DEPTH: 2 
		Tree: {0: (0, 1, 2), 1: (0, 1, 6), 3: (0, 0, 1), 8: (1, 1, 1.0), 4: (0, 1, 4), 9: (1, 0, 1.0), 10: (1, 1, 1.0), 2: (0, 0, 1), 6: (0, 0, 1), 14: (1, 0, 1.0)} [0 0 0 0 0 1 1 1 1 1]
DEPTH: 3 
		Tree: {0: (0, 1, 2), 1: (0, 1, 6), 3: (0, 0, 1), 8: (0, 0, 1), 18: (1, 1, 1.0), 4: (0, 1, 4), 9: (0, 0, 1), 20: (1, 0, 1.0), 10: (0, 0, 1), 22: (1, 1, 1.0), 2: (0, 0, 1), 6: (0, 0, 1), 14: (0, 0, 1), 30: (1, 0, 1.0)} [0 0 0 0 0 1 1 1 1 1]
DEPTH: 4 
		Tree: {0: (0, 1, 2), 1: (0, 1, 6), 3: (0, 0, 1), 8: (0, 0, 1), 18: (0, 0, 1), 38: (1, 1, 1.0), 4: (0, 1, 4), 9: (0, 0, 1), 20: (0, 0, 1), 42: (1, 0, 1.0), 10: (0, 0, 1), 22: (0, 0, 1), 46: (1, 1, 1.0), 2: (0, 0, 1), 6: (0, 0, 1), 14: (0, 0, 1), 30: (0, 0, 1), 62: (1, 0, 1.0)} [0 0 0 0 0 1 1 1 1 1]


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine.
Для этого используем numpy.

In [10]:
%time clf.fit(X_train, y_train)

CPU times: user 3.55 ms, sys: 1.46 ms, total: 5.02 ms
Wall time: 6.06 ms


DecisionTreeClassifier()

In [9]:
%time my_clf.fit(X_train, y_train)

CPU times: user 448 ms, sys: 2.52 ms, total: 450 ms
Wall time: 453 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match).

Данные и описания колонок во вложениях.

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Либо воспользоваться функцией:

In [11]:
def preprocess_spd_data(df):
    df = df.iloc[:, :97]

    to_drop = [
        'id', 'idg', 'condtn', 'round', 'positin1', 'race_o', 'pf_o_att',
        'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
        'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'prob_o','met_o',
        'field', 'zipcode', 'expnum', 'mn_sat', 'tuition'
    ]

    df = df.drop(to_drop, axis=1)
    df = df.dropna(subset=['age', 'imprelig', 'imprace', 'date'])

    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)

    # attr1 processing
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                                            'amb1_1', 'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] =\
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100

    # attr2 processing
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                                            'amb2_1', 'shar2_1']].sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] =\
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)

    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]

        if i != 4:
            feat.remove('shar{}_1'.format(i))

        df = df.drop(feat, axis=1)

    df = df.drop(['wave'], axis=1)
    df = df.dropna()
    return df

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. В качестве метрики можно взять roc-auc.


In [12]:
df = pd.read_csv('Speed_Dating_Data.csv', encoding='latin1')

In [13]:
data = preprocess_spd_data(df)

In [14]:
y = data['match']
X = data.drop(['match'], axis=1).to_numpy()

In [15]:
my_clf = MyDecisionTreeClassifier()
my_clf.fit(X, y)

In [16]:
print('roc-auc = ', roc_auc_score(y_true=y, y_score=my_clf.predict(X)))

roc-auc =  0.672108495787467


Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности.
Постройте графики зависимости точности на валидации и трейне от глубины дерева, от минимального числа объектов для сплита. (Т.е должно быть 2 графика, на каждой должны быть 2 кривые - для трейна и валидации)
Какой максимальной точности удалось достигнуть?

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=30)

In [ ]:
my_clf_miscl = MyDecisionTreeClassifier(criterion='misclassification')
my_clf_ent = MyDecisionTreeClassifier(criterion='entropy')
my_clf_gini = MyDecisionTreeClassifier(criterion='gini')
my_clf_miscl.fit(X_train, y_train)
my_clf_ent.fit(X_train, y_train)
my_clf_gini.fit(X_train, y_train)

print('roc-auc_score on valid', roc_auc_score(y_true=y_test, y_score=my_clf_miscl.predict(X_test)))
print('roc-auc_score on valid', roc_auc_score(y_true=y_test, y_score=my_clf_ent.predict(X_test)))
print('roc-auc_score on valid', roc_auc_score(y_true=y_test, y_score=my_clf_gini.predict(X_test)))

Известным фактом является то, что деревья решений сильно переобучаются при увеличении глубины и просто запоминают трейн.
Замечаете ли вы такой эффект судя по графикам? Что при этом происходит с качеством на валидации?

In [ ]:
plt.figure(figsize=(16, 9))
plt.title('depth influence')
train = []
test = []
for i in range(1, 11):
    my_clf = MyDecisionTreeClassifier(max_depth=i)
    my_clf.fit(X_train, y_train)
    roc_auc_train = roc_auc_score(y_true=y_train, y_score=my_clf.predict(X_train))
    roc_auc_test = roc_auc_score(y_true=y_test, y_score=my_clf.predict(X_test))
    train.append(roc_auc_train)
    test.append(roc_auc_test)
plt.plot(range[1, 11], train, label='train')
plt.plot(range[1, 11], test, label='test')
plt.legend()
plt.xlabel('max_depth')
plt.ylabel('metric')
plt.show()

In [ ]:
plt.figure(figsize=(16, 9))
plt.title('depth influence')
train = []
test = []
for i in range(2, 11):
    my_clf = MyDecisionTreeClassifier(min_samples_split=i, max_depth=8)
    my_clf.fit(X_train, y_train)
    roc_auc_train = roc_auc_score(y_true=y_train, y_score=my_clf.predict(X_train))
    roc_auc_test = roc_auc_score(y_true=y_test, y_score=my_clf.predict(X_test))
    train.append(roc_auc_train)
    test.append(roc_auc_test)
plt.plot(range[1, 11], train, label='train')
plt.plot(range[1, 11], test, label='test')
plt.legend()
plt.xlabel('max_depth')
plt.ylabel('metric')
plt.show()

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве.

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший.
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева.

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [ ]:
my_clf = MyDecisionTreeClassifier()
my_clf.fit(X_train, y_train)

In [ ]:
imp_feature = my_clf.get_feature_importance()
top_features = sorted(imp_feature.items(), key= lambda item: item[1], reverse=True)[:10]
for i in range(10):
    print(f'feature {i + 1}: {X_train.columns[top_features[i][0]]}, gain = {top_features[i][i]}')

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

